# Projeto: Análise do Mercado Imobiliário em Lisboa com Dados do Airbnb

Este projeto tem como objetivo explorar e analisar dados públicos de imóveis disponíveis no Airbnb na cidade de Lisboa. Por meio de técnicas de análise exploratória de dados (EDA) e visualizações, buscamos identificar padrões de preços, tipos de acomodações, regiões com maior disponibilidade e insights que ajudem a entender melhor o mercado de aluguel de curto prazo na cidade.

**Autor:** Lisandro Almeida Viana

🔍 **Problemas que este projeto ajuda a responder:**
- Qual o preço médio dos imóveis por tipo de acomodação?
- Quais são os bairros com maior oferta de imóveis e disponibilidade?
- Quais fatores influenciam mais no preço de um imóvel no Airbnb?
- Como um novo anunciante pode se posicionar melhor no mercado?
- É possível prever o preço de um imóvel com base em suas características?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## 🎯 Objetivo da Análise
- Entender a distribuição de preços dos imóveis
- Comparar valores por tipo de acomodação
- Avaliar a disponibilidade por região
- Identificar bairros com maior oferta

In [ ]:
df = pd.read_csv('/mnt/data/listings_amostra.csv')

In [ ]:
colunas = [
    'name', 'neighbourhood_cleansed', 'latitude', 'longitude',
    'room_type', 'price', 'minimum_nights', 'availability_365',
    'number_of_reviews', 'review_scores_rating']
df = df[colunas]

In [ ]:
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df.dropna(inplace=True)
df = df[(df['price'] > 0) & (df['minimum_nights'] < 365)]

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['price'], bins=50, kde=True)
plt.title('Distribuição de Preços dos Imóveis no Airbnb (Lisboa)')
plt.xlabel('Preço (€)')
plt.ylabel('Quantidade de imóveis')
plt.xlim(0, 1500)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='room_type', y='price', data=df)
plt.title('Preço por Tipo de Acomodação')
plt.xticks(rotation=45)
plt.ylim(0, 1500)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
df_grouped = df.groupby('neighbourhood_cleansed')['availability_365']
.mean().sort_values(ascending=False).head(10)
sns.barplot(x=df_grouped.values, y=df_grouped.index)
plt.title('Top 10 bairros com maior disponibilidade média anual')
plt.xlabel('Dias disponíveis por ano')
plt.ylabel('Bairro')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Mapa de Correlação entre Variáveis')
plt.show()

In [ ]:
print("\n🔍 Principais insights:")
print("- Correlação moderada entre review_scores_rating e preço")
print("- Número de noites mínimas não influencia muito o preço")
print("- Número de avaliações pode ter relação com disponibilidade e localização")

In [ ]:
df_model = df[['price', 'minimum_nights', 'availability_365', 'number_of_reviews', 'review_scores_rating']]
X = df_model.drop('price', axis=1)
y = df_model['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("\n📊 Resultados da Regressão Linear:")
print("R² Score:", r2_score(y_test, y_pred))
print("Erro Quadrático Médio (RMSE):", mean_squared_error(y_test, y_pred, squared=False))